#### Imported necessary libraries

In [21]:
from email.parser import HeaderParser
import nltk
import pandas as pd
import re
from bs4 import BeautifulSoup
import joblib
from Parsers.EmailParser import emailParser
from DTO.EmailData import EmailData

#### Pandas ENV Variables

In [17]:
pd.set_option('display.max_colwidth', 200)

#### Transform the email headers from string to pandas DataFrame

In [33]:
e = "Received: from VE1P189MB0912.EURP189.PROD.OUTLOOK.COM (2603:10a6:800:168::7)\r\n by DB9P189MB1833.EURP189.PROD.OUTLOOK.COM with HTTPS; Wed, 17 Apr 2024\r\n 21:14:17 +0000\r\nARC-Seal: i=2; a=rsa-sha256; s=arcselector9901; d=microsoft.com; cv=pass;\r\n b=QiiUpCUxiYQt7wwfUkXzSejgljRvjg511A8L6Q3p2BIRRmThBfBFgakHe6dS4zD9Qkl2/qu+oZqJms6QaWsdi1ffkB3X4f0MPtqldJopmYKJ/HsaDR8eFVKFYHkTohYEeWfllhIWK+fdrTDjUbLojrWtafSR3ytLsLAJTi5KciVZT0z5oD015o1oFHbmYL060I1ojcSviuBiHKFo4Ko/u4UCXJ2cUjB7vBDnWWes85r2DxMxC0pby8WUxzJJxaKx/66keTrSwEwwZZ2vrzSeiow/7nNd2U1nEqPRKPJoWpefxMG+Fu90bNUCPM6ZTh//v+vAHZHboG7kR6z460a4Mw==\r\nARC-Message-Signature: i=2; a=rsa-sha256; c=relaxed/relaxed; d=microsoft.com;\r\n s=arcselector9901;\r\n h=From:Date:Subject:Message-ID:Content-Type:MIME-Version:X-MS-Exchange-AntiSpam-MessageData-ChunkCount:X-MS-Exchange-AntiSpam-MessageData-0:X-MS-Exchange-AntiSpam-MessageData-1;\r\n bh=vpcJS4kYx3N/+N8BliNOZvIzek/Wd7EQ7+oUE/R4FzU=;\r\n b=Iw8U1ldH7df+0d1dOQXYxwZOOE3IN+Gg2PEwjwcdZl1eHPFY9vEWaE0YtsJZJ3OYOFegjQ/kTmpx/AGoN++0KG+hcCAHm//otkX61BOmsBYx+NjUFtYUcWzNKWIoNX2sqx4Z6RT6yrLnoSntneao1gQlDCAdhDWSo8HgUKM17uBRIbse+ukt1OvLKjfQXI0M+S7qA8JqE0igISllZGrK8I4T8Wh2kqDOi555nff7VqvfrvIdqud7CfRLZQ7gtugequgx5KhDszKFDq9VE7ohuYRg78aT6ylkM2YFdErAfjDAU/PAocMy9aiDxma7Y+g+tAnbvBGnCdjXziyGOYwM6Q==\r\nARC-Authentication-Results: i=2; mx.microsoft.com 1; spf=pass (sender ip is\r\n 40.107.243.115) smtp.rcpttodomain=outlook.com\r\n smtp.mailfrom=accountprotection.microsoft.com; dmarc=pass (p=reject sp=reject\r\n pct=100) action=none header.from=accountprotection.microsoft.com; dkim=pass\r\n (signature was verified) header.d=accountprotection.microsoft.com; arc=pass\r\n (0 oda=0 ltdi=1)\r\nReceived: from DU2PR04CA0344.eurprd04.prod.outlook.com (2603:10a6:10:2b4::17)\r\n by VE1P189MB0912.EURP189.PROD.OUTLOOK.COM (2603:10a6:800:168::7) with\r\n Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.7472.39; Wed, 17 Apr\r\n 2024 21:14:16 +0000\r\nReceived: from DB3PEPF00008859.eurprd02.prod.outlook.com\r\n (2603:10a6:10:2b4:cafe::4d) by DU2PR04CA0344.outlook.office365.com\r\n (2603:10a6:10:2b4::17) with Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.7495.26 via Frontend\r\n Transport; Wed, 17 Apr 2024 21:14:15 +0000\r\nAuthentication-Results: spf=pass (sender IP is 40.107.243.115)\r\n smtp.mailfrom=accountprotection.microsoft.com; dkim=pass (signature was\r\n verified) header.d=accountprotection.microsoft.com;dmarc=pass action=none\r\n header.from=accountprotection.microsoft.com;compauth=pass reason=100\r\nReceived-SPF: Pass (protection.outlook.com: domain of\r\n accountprotection.microsoft.com designates 40.107.243.115 as permitted\r\n sender) receiver=protection.outlook.com; client-ip=40.107.243.115;\r\n helo=NAM12-DM6-obe.outbound.protection.outlook.com; pr=C\r\nReceived: from NAM12-DM6-obe.outbound.protection.outlook.com (40.107.243.115)\r\n by DB3PEPF00008859.mail.protection.outlook.com (10.167.242.4) with Microsoft\r\n SMTP Server (version=TLS1_2, cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id\r\n 15.20.7452.22 via Frontend Transport; Wed, 17 Apr 2024 21:14:15 +0000\r\nX-IncomingTopHeaderMarker:\r\n OriginalChecksum:17C72A35C7BA048A8D732BA3BC196E22E4E6F7D9BE2C4618FC94C1A25F41009C;UpperCasedChecksum:88EF6AF15B7F00A8192C591A5F3A4B4A1625524FA798D39CA554755E747E58AF;SizeAsReceived:6304;Count:38\r\nARC-Seal: i=1; a=rsa-sha256; s=arcselector9901; d=microsoft.com; cv=none;\r\n b=i5xa+eh5Chfli9gpOG2KxKk4TRIjENWLZfzqigmFAaiNyihxSIOgH469BneVxvzA5EyaLCgdqzVMvDjCNt64agSznebOisr/dmjbW22Gn3ybjdp5Kidmgn/JSboYWy2sSiIbNeqIwZF5eMY8pyImG8kEXLNoHroJvL2+V4FyT7suDmYxpOY0Yzlcf9aO2yPKGdhQJnQnS6QPoPUCkxrmSAmUvXdJu44vq5PN1cO+gdg/7ek2HRTwZBBrNYTX2SQxkea5HERmdbW3XU2Qca5GOF3jkejfqB5yZu8QHGtnO5Ays9DVbGQvvHfUBoPxqZXHXHiIx8ch2377LwUyVwHWmQ==\r\nARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=microsoft.com;\r\n s=arcselector9901;\r\n h=From:Date:Subject:Message-ID:Content-Type:MIME-Version:X-MS-Exchange-AntiSpam-MessageData-ChunkCount:X-MS-Exchange-AntiSpam-MessageData-0:X-MS-Exchange-AntiSpam-MessageData-1;\r\n bh=vpcJS4kYx3N/+N8BliNOZvIzek/Wd7EQ7+oUE/R4FzU=;\r\n b=ccP1Bc2vmlbDls884xO/buS5NNC6m1ansopbrTEbqBiGCQ4wTTZa4fKZw8hyjyahLieCIwBnuPBDg5OH7FbGvm/7EsxSRN/VBoPDAUwfhkxfHK/t8h7XgSzW0vwzRnYYHjOxNhdQDnurwRPsSIiHO+P9gVNIryegzv/ntMev7R0nvGAqr6MYYinsWUTey2pPQQ1OMQFqVBv+xu/pbmMNXVtpfzIPGsFpXq/VQdaJyDl/Vule+C0GrKcfN7xpoQA3fv3PzXZJN6HfvqI3bj3JOiKelAMvcViFA49VSTx02cGNKjj6hotv7XzKOk01KtrHgw5YOFHgQC1XlUkFJClRBg==\r\nARC-Authentication-Results: i=1; mx.microsoft.com 1; spf=none; dmarc=none\r\n action=none header.from=accountprotection.microsoft.com; dkim=none (message\r\n not signed); arc=none\r\nDKIM-Signature: v=1; a=rsa-sha256; c=relaxed/relaxed;\r\n d=accountprotection.microsoft.com; s=selector1;\r\n h=From:Date:Subject:Message-ID:Content-Type:MIME-Version:X-MS-Exchange-SenderADCheck;\r\n bh=vpcJS4kYx3N/+N8BliNOZvIzek/Wd7EQ7+oUE/R4FzU=;\r\n b=n2boFIGxQp50TJHT1PTfJyLOh8Elhu/HuM/tx5birFgzyg961nKhWCYo6ToxvLA9eozs2PJq//XL4STwzMmThYAWO03nM46xk35H7Grj5haSupj2+guzREKVS1K3v+SAqA8rwZHkoyfKn+EdJZ4NgqrzPTIrmsRBYo9Jqo5sskg=\r\nReceived: from CH2PR07CA0040.namprd07.prod.outlook.com (2603:10b6:610:5b::14)\r\n by LV8PR16MB6218.namprd16.prod.outlook.com (2603:10b6:408:1cf::16) with\r\n Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.7472.39; Wed, 17 Apr\r\n 2024 21:14:14 +0000\r\nReceived: from CH2PEPF0000009A.namprd02.prod.outlook.com\r\n (2603:10b6:610:5b:cafe::3b) by CH2PR07CA0040.outlook.office365.com\r\n (2603:10b6:610:5b::14) with Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.7472.33 via Frontend\r\n Transport; Wed, 17 Apr 2024 21:14:14 +0000\r\nX-MS-Exchange-Authentication-Results: spf=none (sender IP is 52.188.222.33)\r\n smtp.mailfrom=accountprotection.microsoft.com; dkim=none (message not signed)\r\n header.d=none;dmarc=none action=none\r\n header.from=accountprotection.microsoft.com;\r\nReceived: from accountprotection.microsoft.com (52.188.222.33) by\r\n CH2PEPF0000009A.mail.protection.outlook.com (10.167.244.22) with Microsoft\r\n SMTP Server (version=TLS1_2, cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id\r\n 15.20.7452.22 via Frontend Transport; Wed, 17 Apr 2024 21:14:14 +0000\r\nFrom: Microsoft account team\r\n\t<account-security-noreply@accountprotection.microsoft.com>\r\nDate: Wed, 17 Apr 2024 14:14:13 -0700\r\nSubject: New app(s) connected to your Microsoft account\r\nTo: testaddin2024@outlook.com\r\nX-Priority: 3\r\nX-MSAPipeline: MessageDispatcherEOP\r\nMessage-ID: <JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>\r\nX-MSAMetaData:\r\n =?us-ascii?q?DpWQFOjMoVrMlU7bG57UhdEOpdlQbCYyV88VlLpa!oGS3rZat9TMFbtCtchtr?=\r\n =?us-ascii?q?aarUNSXKleIBoWpUQJ5SsORhqby3V4qcTDdLOWwe2sy475XW8AEgE0JbNGMNB?=\r\n =?us-ascii?q?6Cd4DFRA$$?=\r\nMIME-Version: 1.0\r\nContent-Type: multipart/alternative; boundary=\"=-r3iB+mxalPL1yojUv8860w==\"\r\nReturn-Path: account-security-noreply@accountprotection.microsoft.com\r\nX-MS-TrafficTypeDiagnostic:\r\n CH2PEPF0000009A:EE_FirstParty-MicrosoftAccount-V3-System|LV8PR16MB6218:EE_FirstParty-MicrosoftAccount-V3-System|DB3PEPF00008859:EE_|VE1P189MB0912:EE_|DB9P189MB1833:EE_\r\nX-MS-Office365-Filtering-Correlation-Id: f433591a-1c99-46ef-0c2d-08dc5f235676\r\nX-MS-Exchange-SenderADCheck: 1\r\nX-MS-Exchange-AntiSpam-Relay: 0\r\nX-Microsoft-Antispam-Untrusted: BCL:0;\r\nX-Microsoft-Antispam-Message-Info-Original:\r\n HV9N/mIdKhXYUPElCUeTVqHIRaM4POPfvK4Z1qV8eH6ezeGjhaZ7R8Klr2gCIYxysni/l5NECu5kgQEX42F4m8W+9WqH9dcPvDF/eoJLvrbcxByWDbZhOISD2IkFoj8YJsZ/mEA0xyeBFN1LK0vzm4UfsE0t/VJR0PGCQ1/+MIH2jqZ4asYqZtqf+iQQ0Zqku0UXPgtVAYKj5X8Nlj5O11KWaV43Ov7QEZmUe/vLWv/uLTpmUX9InBq3fXbjlpaJF6F+YICByZZ9LC3nT7crJ8cJseT1G0G6Wn/B7Ve8JZYQysifEgVqXq/6TfXLu6PKImW1pK4DDhTo+iMLpQw1muttHjqf8UvBLgOChdVWSc5pdutNMgupjaO0Zjx6xy8ljyRhwCQlDedHqikTCcG2tJ4q+g3sLJ2RxSsQaKgCfoBhBbYp9zqaGRSAh8R80gMdzj8f/ITBh9LfkJjl4GI25oq30pUK2w6SYG2mpEqWm5zvSswexJc0ekYajlMcxKb9dPH5lY8fbF7itFdrR07Ul3jw1/5uLCWOwaT0xeqvLY+4SX5WUnA1uPYl+oAKpjKbZPf6vGaVmKmYlS0+ES9A9rC23wP8qFJoE4aIhj7OZpxqmuLxcel5q/fdN0nPld9Gf2cP3XNnsUVRvclyePh/HAyJx/jNb7o+Wp10u/nkvspXz5H45MllMuYaMN1iVioE6AdY4IA1yZg2PSrPNjsnQ7R907N924XvH+Ko0BIKYUhPyzu6p5qVPBtdZ+p9y2zC\r\nX-Forefront-Antispam-Report-Untrusted:\r\n CIP:52.188.222.33;CTRY:US;LANG:en;SCL:1;SRV:;IPV:NLI;SFV:NSPM;H:accountprotection.microsoft.com;PTR:messagedispatchervip.MSAEAP-Prod-BL02P.BL02P.ap.gbl;CAT:NONE;SFS:(13230031)(376005)(61400799018);DIR:OUT;SFP:1102;\r\nX-MS-Exchange-AntiSpam-MessageData-Original-ChunkCount: 1\r\nX-MS-Exchange-AntiSpam-MessageData-Original-0:\r\n 0yx5Wt6vJM4CYZqk15P3Y4qW8P8qzXFXUbkgh6xoxeG7zVGNg0tAvcxjeAzhgbI1u+D2Ipu3W9C+uPNndaVSRhbMrQRxQC1Hq5InZV4GIeMU5GBd772dPyEbFxXazEA9GswcON4anzkbKLqM52uDKPn1pRMmGsp357KqNubi2yczIcwyrhBULHC3+U3kBoFeG0Ja5MeRVxmYCEE2PeAgIVR3cG2iudcP2rxIMg43avsLkqZ4LxLgMgD7iANp0prWHRFASAu20ihU5B/RfSfEZPk+Sk82V/YNhVLxdfwsaGVef/o0Uc56oY9x2DWGrTDM6hmeqOBDbYJBiIDBCWL4ExHF15DZ8u1RruHmVzHeKn/1h1BOJOsNHNqlG93iTrBSeZtHVCod83Vs7iqJsk31pBuDvbY44b1yWUHo9QZVCzoa1bqpynntaNqVweWJcq9teZswUfJFWmMLgHGaVKKZz8J1oGMjt89O5Tb2IoyFgHlx0uTIMUmke7ElO+5nrMsoHlw5Ohrj1JovYVFl6CqTLA==\r\nX-MS-Exchange-Transport-CrossTenantHeadersStamped: LV8PR16MB6218\r\nX-IncomingHeaderCount: 38\r\nX-MS-Exchange-Organization-ExpirationStartTime: 17 Apr 2024 21:14:15.8838\r\n (UTC)\r\nX-MS-Exchange-Organization-ExpirationStartTimeReason: OriginalSubmit\r\nX-MS-Exchange-Organization-ExpirationInterval: 1:00:00:00.0000000\r\nX-MS-Exchange-Organization-ExpirationIntervalReason: OriginalSubmit\r\nX-MS-Exchange-Organization-Network-Message-Id:\r\n f433591a-1c99-46ef-0c2d-08dc5f235676\r\nX-EOPAttributedMessage: 0\r\nX-EOPTenantAttributedMessage: 84df9e7f-e9f6-40af-b435-aaaaaaaaaaaa:0\r\nX-MS-Exchange-Organization-MessageDirectionality: Incoming\r\nX-MS-Exchange-Transport-CrossTenantHeadersStripped:\r\n DB3PEPF00008859.eurprd02.prod.outlook.com\r\nX-MS-Exchange-Transport-CrossTenantHeadersPromoted:\r\n DB3PEPF00008859.eurprd02.prod.outlook.com\r\nX-MS-PublicTrafficType: Email\r\nX-MS-Exchange-Organization-AuthSource:\r\n DB3PEPF00008859.eurprd02.prod.outlook.com\r\nX-MS-Exchange-Organization-AuthAs: Anonymous\r\nX-MS-UserLastLogonTime: 4/17/2024 9:13:05 PM\r\nX-MS-Office365-Filtering-Correlation-Id-Prvs:\r\n 5cbd8751-9983-4892-3ce1-08dc5f23556d\r\nX-MS-Exchange-EOPDirect: true\r\nX-Sender-IP: 40.107.243.115\r\nX-SID-PRA: ACCOUNT-SECURITY-NOREPLY@ACCOUNTPROTECTION.MICROSOFT.COM\r\nX-SID-Result: PASS\r\nX-MS-Exchange-Organization-PCL: 2\r\nX-MS-Exchange-Organization-SCL: 1\r\nX-Microsoft-Antispam: BCL:2;\r\nX-MS-Exchange-CrossTenant-OriginalArrivalTime: 17 Apr 2024 21:14:15.7432\r\n (UTC)\r\nX-MS-Exchange-CrossTenant-Network-Message-Id: f433591a-1c99-46ef-0c2d-08dc5f235676\r\nX-MS-Exchange-CrossTenant-Id: 84df9e7f-e9f6-40af-b435-aaaaaaaaaaaa\r\nX-MS-Exchange-CrossTenant-OriginalAttributedTenantConnectingIp: TenantId=5ba90553-c2cd-460e-b5fd-ab93ad9155c7;Ip=[52.188.222.33];Helo=[accountprotection.microsoft.com]\r\nX-MS-Exchange-CrossTenant-AuthSource:\r\n DB3PEPF00008859.eurprd02.prod.outlook.com\r\nX-MS-Exchange-CrossTenant-AuthAs: Anonymous\r\nX-MS-Exchange-CrossTenant-FromEntityHeader: Internet\r\nX-MS-Exchange-CrossTenant-RMS-PersistedConsumerOrg:\r\n 00000000-0000-0000-0000-000000000000\r\nX-MS-Exchange-Transport-CrossTenantHeadersStamped: VE1P189MB0912\r\nX-MS-Exchange-Transport-EndToEndLatency: 00:00:01.3081534\r\nX-MS-Exchange-Processed-By-BccFoldering: 15.20.7452.029\r\nX-Message-Info:\r\n\tqZelhIiYnPnnjsbBf6asGz6sLW/nE8VmwLFBaXurAfZe3ppWmkedikChkUc8tpabFLoTgl9+ydPtdcuAFcU/dDsjCKsSXhGlqo7XzxR9Zv184nT47cZGragnw0Y4W2X/NKG9C0z/4g2qV8Pz8WMAM85nxbRSHzy7/CKqdLR3YH55YZ8MjKQeuU+BOVOXKvswfvTmTthmY5c=\r\nX-Message-Delivery: Vj0xLjE7dXM9MDtsPTA7YT0xO0Q9MTtHRD0xO1NDTD0z\r\nX-Microsoft-Antispam-Mailbox-Delivery:\r\n\tucf:0;jmr:0;ex:0;auth:1;dest:I;ENG:(5062000305)(920221119095)(90000117)(920221120095)(90012020)(91020020)(91040095)(9050020)(9100341)(944500132)(4810010)(4910033)(9575002)(10195002)(9320005)(120001);\r\nX-Microsoft-Antispam-Message-Info:\r\n\t=?us-ascii?Q?fq1NnuRZljvxQCPlkgVih8faufrUDZf0438WQXUfc493ayvow+ed0f3slYko?=\r\n =?us-ascii?Q?xlUCs8/oh0rlEZgnapYxkVzw3j4uJjiG3fzG9zsQc7KukeGkTDhIHHWI2JVJ?=\r\n =?us-ascii?Q?3fHa9cIAJSmH2/dxoZsi03wdkKdycOYKWLRRSvnYCi+fe2el/PMXHCeZXm4Z?=\r\n =?us-ascii?Q?0FnXp/Cqw8Pa9EgjrjFdI230B15dTjY7Wjl9OVkwANXj/us8nH0GzlTNY+IT?=\r\n =?us-ascii?Q?MSUCgoHQcIId1s8kjBcTa1jv8BSPIhy5sArMQ5RFv24+pltFAnnCno1iPrUA?=\r\n =?us-ascii?Q?rMfLOI6647j7fj/CEUwcfB9ZyAVSQv4FJLKuM/JxrFZc36gGm+WIBAXiAouW?=\r\n =?us-ascii?Q?F2PWKCWaMU/qsyiBnwuSBWnEWh/6h3kBQRfU9K0AOlCvcL08ViZhUX4Np2WT?=\r\n =?us-ascii?Q?IcNw42A4Y0JQwRQukQhtNrUZv95OZTS0Nc34rh4Zr+XqylZ6Ke5lu+jN3Nii?=\r\n =?us-ascii?Q?B0lBi4drd33DwZY5728Lh2JLjg9zWiiL5Xl66s5sS+9qHAKgasCbZdC306QX?=\r\n =?us-ascii?Q?SsU6A8W8+195zUBt4yDagImfLeXZpeAzOvxWZjvdE3/pXg6fL4GbxicvJL6s?=\r\n =?us-ascii?Q?6woURnoFWIc3CggZ8470qmVjD4wnVi0EJzLY+SVuK8E33CZ9jLJ6ttn86dLe?=\r\n =?us-ascii?Q?vK2Hs4tl3RlBTTPRNeJQkLX7Pc1cQEROyUpFOhfZlsRlQGqF/8mfQ6Q+pJd2?=\r\n =?us-ascii?Q?7JefOSBS3Y9thQbWKTtFJiY9l76KPFH78qyY11FR8Z5/Aoq8j4CTjOcEb/rS?=\r\n =?us-ascii?Q?q0BccDtwnjTzrX/KRIG5ondHo+HDxRGQ6S4icZ6zDc3r2fTfT3YP0EQFi+mG?=\r\n =?us-ascii?Q?YRSCrJa11yOAZiJJhcoGzQFh63lmpPO2UVU4tINOq/ZgVVdUaDwioCouwGgy?=\r\n =?us-ascii?Q?kYun+P3/VO54q2ZYgAITMCsRWb9xXafPSwoknbZbIo68Uhg+Q97IM/9WcaKk?=\r\n =?us-ascii?Q?t0muvxS8clgjoY6sGpFuMoP15L3olgEjsdxeeyVKfcQrWHSQ6jPsEy7DSDkG?=\r\n =?us-ascii?Q?y0LizEyOtNj1pPDecyGHm9oNmylYu7UERXu6HIP5iU83nXB5eX9LPfwDofJS?=\r\n =?us-ascii?Q?OsdI3QdqJPE3tKvn/6tvcg6KP4zmb6Y+2XJEac+7/iKRN2VG2WbnSDf5X6H3?=\r\n =?us-ascii?Q?v5nV8wV0XpK9UVe75uPC4+fFx4COrxPMso/OtpX+V7V8tA3VsaEG9/y2P/7/?=\r\n =?us-ascii?Q?MTcs/N6EKrWw1jOgAlGs7KaAFoSkSddJZ+vjG+wCZRc209GdsH582FtySdQS?=\r\n =?us-ascii?Q?xyA52svI3ltsNHteu1wEhfUc4MhFo8cfabNDfnPkLVO3e+KLfz1Al4ufkCfA?=\r\n =?us-ascii?Q?HYZBc+gMGzdmrRr8m109GKM+YfKaGrRRPBNCpViBPx82O0Dtvc+YeQkFkkyv?=\r\n =?us-ascii?Q?5cBeds/a7NX127ofErRIFEcO/EAkSfP79/LcI/mJ/W7UJSshxF8Wzh29+Ifu?=\r\n =?us-ascii?Q?+gUjf69KVuSXlHCi4O9JYNrxxany/N7hjeoJFAEPgJC+xeUE43nASOe4dGX8?=\r\n =?us-ascii?Q?hLaTaMftZ6i/wl0LnJuYh2m5gJOi4Qt7l6cMjt+vdEil/KMdXa+4R7tSm+6x?=\r\n =?us-ascii?Q?uaquIERsSNnlH4cVFsi9MNGVSJogqrcC8YCShjfeJaxEXnurRDNzqUW2LrCN?=\r\n =?us-ascii?Q?6CFcSnC6nMqNnzjysVSxVqX/9fQZfK8kmLqAMcDiJ+CvvfkZrqrhBT1DH8RF?=\r\n =?us-ascii?Q?BRLA6UoUW8mheTGbEiYLPHf6DUOpjJxmuen/Jy4capTs/evo9cft3Cumizyb?=\r\n =?us-ascii?Q?NJx9ykEIx+e64mjPLC7gs086ZjDzd+0=3D?=\r\n"

body = "<style type=\"text/css\">\r\n<!--\r\n-->\r\n</style><div class=\"apiBody\"><div><table dir=\"ltr\"><tbody><tr><td id=\"x_i1\" style=\"padding:0; font-family:'Segoe UI Semibold','Segoe UI Bold','Segoe UI','Helvetica Neue Medium',Arial,sans-serif; font-size:17px; color:#707070\">Microsoft account</td></tr><tr><td id=\"x_i2\" style=\"padding:0; font-family:'Segoe UI Light','Segoe UI','Helvetica Neue Medium',Arial,sans-serif; font-size:41px; color:#2672ec\">New app(s) have access to your data</td></tr><tr><td id=\"x_i3\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">Graph Explorer connected to the Microsoft account <a dir=\"ltr\" id=\"x_iAccount\" class=\"x_link\" href=\"mailto:te**4@outlook.com\" style=\"color:#2672ec; text-decoration:none\">te**4@outlook.com</a>.</td></tr><tr><td id=\"x_i4\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">If you didn't grant this access, please remove the app(s) from your account.</td></tr><tr><td style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\"><table border=\"0\" cellspacing=\"0\"><tbody><tr><td bgcolor=\"#2672ec\" style=\"background-color:#2672ec; padding-top:5px; padding-right:20px; padding-bottom:5px; padding-left:20px; min-width:50px\"><a id=\"x_i5\" href=\"https://account.live.com/consent/Manage?fn=email\" style=\"font-family:'Segoe UI Semibold','Segoe UI Bold','Segoe UI','Helvetica Neue Medium',Arial,sans-serif; font-size:14px; text-align:center; text-decoration:none; font-weight:600; letter-spacing:0.02em; color:#fff\">Manage your apps</a></td></tr></tbody></table></td></tr><tr><td id=\"x_i6\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">You can also <a id=\"x_iLink3\" class=\"x_link\" href=\"https://account.live.com/SecurityNotifications/Update\" style=\"color:#2672ec; text-decoration:none\">opt out</a> or change where you receive security notifications.</td></tr><tr><td id=\"x_i7\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">Thanks,</td></tr><tr><td id=\"x_i8\" style=\"padding:0; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">The Microsoft account team</td></tr></tbody></table><div lang=\"en\" style=\"margin-top:20px; margin-bottom:10px\"><a class=\"x_link\" href=\"https://go.microsoft.com/fwlink/?LinkId=521839\">Privacy Statement</a><div style=\"margin-top:10px\">Microsoft Corporation, One Microsoft Way, Redmond, WA 98052</div></div></div></div>"

# body = "<html><head><meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3Dutf-8"></=head><body><p dir=3D"ltr"></p><div class=3D"zd-comment" dir=3D"auto"><p dir==3D"ltr" style=3D"color: #2b2e2f; line-height: 22px; margin: 15px 0">&nbsp;=</p><table style=3D"border-collapse: collapse; font-size: 1em; height: 465p=x; width: 539px" width=3D"539" cellspacing=3D"0" cellpadding=3D"0"><tbody><=tr><td style=3D"border: 1px solid #ddd; padding: 20px 40px 5px; vertical-al=ign: top" align=3D"center" valign=3D"top"><p align=3D"left" dir=3D"ltr" sty=le=3D"color: #2b2e2f; line-height: 22px; margin: 0 0 16px"><img style=3D"he=ight: 50px; max-width: 100%; width: 215px" src=3D"https://altcoinsbox.com/w=p%2Dcontent/uploads/2023/03/full%2Dtrust%2Dwallet%2Dlogo.png" alt=3D"" widt=h=3D"2494" height=3D"582"></p><p align=3D"left" dir=3D"ltr" style=3D"color:= #2b2e2f; line-height: 22px; margin: 0 0 16px"><span style=3D"color: #41414=1; font-size: large">Wallet suspension alert</span></p><p style=3D"backgrou=nd-color: #ffffff; box-sizing: border-box; color: #414141; font-size: 14px;= font-style: normal; font-variant-caps: normal; font-variant-ligatures: nor=mal; font-weight: 400; letter-spacing: normal; line-height: 22px; margin: 0= 0 16px; orphans: 2; text-decoration-color: initial; text-decoration-style:= initial; text-indent: 0px; text-transform: none; white-space: normal; wido=ws: 2; word-spacing: 0px" align=3D"left" dir=3D"ltr">Dear Customer,</p><p s=tyle=3D"background-color: #ffffff; box-sizing: border-box; color: #414141; =font-size: 14px; font-style: normal; font-variant-caps: normal; font-varian=t-ligatures: normal; font-weight: 400; letter-spacing: normal; line-height:= 22px; margin: 0 0 16px; orphans: 2; text-decoration-color: initial; text-d=ecoration-style: initial; text-indent: 0px; text-transform: none; white-spa=ce: normal; widows: 2; word-spacing: 0px" align=3D"left" dir=3D"ltr">Accord=ing to our records, your wallet is still pending verification. The process =of verification can be effortlessly completed through the website provided.=<br>All unauthenticated wallets will be temporarily disabled soon.<br>We ap=ologize for any inconveniences experienced, and kindly note that our aim is= to ensure the safety and satisfaction of our clients.</p><table style=3D"b=ackground-color: #009fc7; border-collapse: separate !important; border-radi=us: 50px; font-size: 1em; height: 29px; letter-spacing: normal; orphans: 2;= text-decoration-color: initial; text-decoration-style: initial; text-size-=adjust: 100%; text-transform: none; widows: 2; width: 224px; word-spacing: =0px" border=3D"0" cellspacing=3D"0" cellpadding=3D"0" bgcolor=3D"#009fc7"><=tbody><tr><td style=3D"border: 1px solid #ddd; font-size: 18px; padding: 18=px; text-size-adjust: 100%; vertical-align: top; width: 184px" align=3D"cen=ter" valign=3D"middle"><a href=3D"https://u596767.ct.sendgrid.net/ls/click?=upn=3DyDgvKZDFDsqJ41SRw-2BU2QfZafOYQphOgvafRq4VGnEX4D0ILZITyVrDJHdclAA5C-2F=wZ3VwFpEQVEZ0DX6cPoZQ-3D-3D1Tjv_470gekIA5U1PugZ-2F-2BY9Ap-2BwpyY5X2yg5Ad6hN=-2FqfBioBqK8L6ifZfvfBBaCNoTsVdcu5qCIQADeYRMTWjbqGQ1NqUz1Kf6sV-2FH06-2Fr198M=OhNfST3TLFsb6CdGucfIGeyVG2aFpH2B7qYHBqhFLiZiRYE9pSnF7iEoIJG6YsiYqwOX3PhhLVL=Ef1fNGkg-2Bx09igkC-2BUzeFeZep5-2Btq91sA-3D-3D" rel=3D"noreferrer"><span sty=le=3D"color: #ffffff">CONFIRM RECOVERY PHRASE</span></a></td></tr></tbody><=/table><p style=3D"background-color: #ffffff; box-sizing: border-box; color=: #414141; font-size: 14px; font-style: normal; font-variant-caps: normal; =font-variant-ligatures: normal; font-weight: 400; letter-spacing: normal; l=ine-height: 22px; margin: 0 0 16px; orphans: 2; text-decoration-color: init=ial; text-decoration-style: initial; text-indent: 0px; text-transform: none=; white-space: normal; widows: 2; word-spacing: 0px" align=3D"left" dir=3D"=ltr">&nbsp;</p><p style=3D"background-color: #ffffff; box-sizing: border-bo=x; color: #414141; font-size: 14px; font-style: normal; font-variant-caps: =normal; font-variant-ligatures: normal; font-weight: 400; letter-spacing: n=ormal; line-height: 22px; margin: 0 0 16px; orphans: 2; text-decoration-col=or: initial; text-decoration-style: initial; text-indent: 0px; text-transfo=rm: none; white-space: normal; widows: 2; word-spacing: 0px" align=3D"left"= dir=3D"ltr">Thank you for choosing&nbsp;trust wallet&nbsp;as your&nbsp; pa=rtner. We're dedicated to making your viewing experience better every day, =.</p></td></tr></tbody></table></div><span style=3D"color:#FFFFFF" aria-hid=den=3D"true">[L7KLJX-35D63]</span></body></html>="

parser = HeaderParser()
h = parser.parsestr(e)

pData = pd.DataFrame(data=h.values(), index=h.keys()).loc[
    ["From", "Subject", "To", "Return-Path", "ARC-Authentication-Results"]]

#Show how the frame looks
print(pData)

body = "aaa"

                                                                            0
From                        Microsoft account team\r\n\t<account-security-...
Subject                        New app(s) connected to your Microsoft account
To                                                  testaddin2024@outlook.com
Return-Path                 account-security-noreply@accountprotection.mic...
ARC-Authentication-Results  i=2; mx.microsoft.com 1; spf=pass (sender ip i...
ARC-Authentication-Results  i=1; mx.microsoft.com 1; spf=none; dmarc=none\...


#### Create the DataFrame that will contain the data for the ML model

In [34]:
def containsWrapper(searchValue: str, text: str) -> int:
    return 1 if searchValue in text else 0


emailDic = {
    "DKIM": 0,                            #Checks whether the email passed the dkim check
    "SPF": 0,                             #Checks whether the email passed the spf check
    "DMARC": 0,                           #Checks whether the email passed the dmarc check
    "ARC": 0,                             #Checks whether the email passed the arc check
    "Body dear word": 0,                  #Checks whether the email has the "dear" word inside it
    "Body form": 0,                       #Checks whether the email has a form html tag 
    "Body html": 0,                       #Checks whether the email contains html
    "Body button": 0,                     #Checks whether the email has buttons
    "Number of Links": 0,                 #Counts the number of links inside the email
    "Body Verify your account": 0,        #Checks whether the email contains the string "verify your account"
    "Body no of function words": 0,       #Counts the number of function words
    "From eq Return": 0,                   #Checks whether the from address is equal to the reply address
    "Message-Id": h["Message-Id"]
}

functionWords = ["Account", "Access", "Bank", "Credit", "Click", "Identity", "Inconvenience", "Information", "Limited",
                 "Log", "Minutes", "Password", "Recently", "Risk", "Security", "Social", "Service", "Suspended",
                 "Urgent"]

#Verify Dkim, spf, arc and dmarc passes
for index, arcMsg in pData.loc["ARC-Authentication-Results"].iterrows():
    if "dkim=pass" in arcMsg[0]:
        emailDic["DKIM"] = 1
    if "spf=pass" in arcMsg[0]:
        emailDic["SPF"] = 1
    if "dmarc=pass" in arcMsg[0]:
        emailDic["DMARC"] = 1
    if "arc=pass" in arcMsg[0]:
        emailDic["ARC"] = 1
        
#Verify body html,form button and verifyAcc        
html, dear, form, button, verifyAcc = containsWrapper("<style", body), containsWrapper("dear", body.lower()), containsWrapper("<form", body), containsWrapper("<button",body), containsWrapper("Verify your account", body.lower())
myValues = [html, dear, form, button, verifyAcc]
myKeys = ["Body html","Body form","Body dear word","Body button","Body Verify your account"]

for x,y in zip(myKeys, myValues):
    emailDic[x] = y

#Verify if "from" addr eq to "return" addr
searchVal = re.search(r'<\s*(.*?)\s*>', str(pData.loc["From"].values))

if searchVal:
    fromAddress = searchVal.group(1)
else:
    fromAddress = "None"
    
if fromAddress == pData.loc["Return-Path"][0]:
    emailDic["From eq Return"] = 1

#Count the number of links by counting the number of href tags in html code
emailDic["Number of Links"] = len(re.findall(r'<a\s+[^>]*href=["\']?(http[^\'" >]+)',body.lower(),re.M))

#Count the number of function words inside the email
for word in functionWords:
    if body.lower().find(word.lower()) != -1:
        emailDic["Body no of function words"] += 1

In [42]:
print(re.search(r'<\s*(.*?)\s*>',"Microsoft account team\r\n\t<account-security-noreply@accountprotection.microsoft.com>").group(1))

account-security-noreply@accountprotection.microsoft.com


In [20]:
string2 = pData.loc["From"].values
print(string2)
string = re.search(r'<(.*?)>', str(pData.loc["From"].values)).group(1)
print(string)

['Microsoft account team\r\n\t<account-security-noreply@accountprotection.microsoft.com>']
account-security-noreply@accountprotection.microsoft.com


In [36]:
def remove_punct(text: str) -> str:
    txt = re.sub('[^a-zA-Z]', ' ', text)
    return txt

def strip_tags(txt: str) -> str:
    s = BeautifulSoup(txt,'html.parser')
    return s.get_text()

def stemming(emailText: str) -> str:
    stopWords = set(nltk.corpus.stopwords.words('english'))
    wordTokens = nltk.word_tokenize(emailText)
    filteredEmailText = [word for word in wordTokens if not word.lower() in stopWords]
    filteredEmailText = ' '.join(filteredEmailText)
    return filteredEmailText

In [37]:
body = stemming(remove_punct(strip_tags(body)))
#Print the data for the model
print(emailDic)

{'DKIM': 1, 'SPF': 1, 'DMARC': 1, 'ARC': 1, 'Body dear word': 0, 'Body form': 0, 'Body html': 0, 'Body button': 0, 'Number of Links': 0, 'Body Verify your account': 0, 'Body no of function words': 0, 'From eq Return': 1, 'Message-Id': '<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>'}


In [38]:
modelData = pd.DataFrame([emailDic])
modelData.set_index('Message-Id', inplace=True)
modelData


,DKIM,SPF,DMARC,ARC,Body dear word,Body form,Body html,Body button,Number of Links,Body Verify your account,Body no of function words,From eq Return
Message-Id,,,,,,,,,,,,
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>,1,1,1,1,0,0,0,0,0,0,0,1


In [39]:
loaded_tfidf = joblib.load('../ML_Model/Tfidf_Vectorizer2.pkl')
text_data = loaded_tfidf.transform([body]).toarray()
text_data = pd.DataFrame(text_data, columns=loaded_tfidf.get_feature_names_out(), index=modelData.index)
all_data = pd.concat([modelData, text_data], axis=1)

In [97]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, <JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB> to <JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>
Columns: 43778 entries, DKIM to zzzzcc
dtypes: float64(43766), int64(12)
memory usage: 342.1+ KB


In [40]:
loaded_model = joblib.load('../ML_Model/LogisticRegression.pkl')

In [41]:
value = loaded_model.predict(all_data)
print(int(value[0]))

1


In [41]:
data = EmailData(
    emailMessage = "aa",
    emailBody = body,
    emailHeaders= e,
    email = "user@example.com"
)
print(emailParser(data))

                                              DKIM  SPF  DMARC  ARC  \
Message-Id                                                            
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>     1    1      1    1   

                                              Body dear word  Body form  \
Message-Id                                                                
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>               0          0   

                                              Body html  Body button  \
Message-Id                                                             
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>          1            0   

                                              Number of Links  \
Message-Id                                                      
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>                3   

                                              Body Verify your account  \
Message-Id                                                               